# Wrangle Data
#### Danny Fay

In [1]:
# Import necessary packages
import os
import numpy as np
import pandas as pd

sc

In [2]:
# Project file path
bus_data_folder = 'data/external/bus_time'

In [3]:
# Define extract buses mapping function
def extractBuses(partId, list_of_records):
    if partId==0:
        list_of_records.next()
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row[7].split('_')) >= 3:
            bus = row[7].split('_')[2][:3]
            if bus in bus_list:
                if len(row[7].split('_')[1].split('-')) > 3:
                    tripId = row[7].split('_')[1].split('-')[3]
                    yield row[0], bus, tripId
                else:
                    tripId = row[7].split('_')[1].split('-')[2]
                    yield row[0], bus, tripId
            

In [4]:
# Download and select bus lines

for month in range(9,11):
    for day in range(9,11):
        
        if month < 10 and day < 10:
            file_name = '/bus_time_2016' + '0' + str(month) + '0' + str(day)
            month_path = '0' + str(month)
        elif month >= 10 and day < 10:
            file_name = '/bus_time_2016' + str(month) + '0' + str(day)
            month_path =  str(month)
        elif month < 10 and day >= 10:
            file_name = '/bus_time_2016' + '0' + str(month)  + str(day)
            month_path =  '0' + str(month)
        elif month >= 10 and day >= 10:
            file_name = '/bus_time_2016' + str(month)  + str(day)
            month_path =  str(month)
            
            
        # Download data    
        os.system('curl -o ' + bus_data_folder + file_name + '.csv.xz ' \
                      'http://data.mytransit.nyc/bus_time/2016/2016-' + month_path + file_name + '.csv.xz')
        # Unzip
        os.system('xz -d ' + bus_data_folder + file_name + '.csv.xz')
        
        # Check if file exists
        if os.path.exists(bus_data_folder + file_name + '.csv'):
            
            # Load into spark rdd
            bus_time = sc.textFile(bus_data_folder + file_name + '.csv', 
                                           use_unicode=True).cache()

            # List of buses for analysis
            bus_list = ['B15', 'B4', 'B3', 'B8', 'B38', 'B43', 'B54', 'B57', 'B62', 'B37', 'B41', 'B11']

            # Extract bus lines matching bus_list
            brooklynBuses = bus_time.mapPartitionsWithIndex(extractBuses)
            
            # Output to csv
            df = pd.DataFrame(brooklynBuses.collect())
            df.to_csv(bus_data_folder + file_name + '.csv')
            
        else:
            pass
            